# News Clustering using KMeans Algorithm
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load data from Pickle 

In [2]:
train = pd.read_pickle("../datastore/1104_00_df.p")
category = pd.read_pickle("../datastore/category.p")
category.head()

,category,name
0,건강·의학 >,건강
1,경제,경제
2,뉴스 > 경제,경제
3,경제 >,경제
4,뉴스 > 경제 > @뉴스룸,경제


In [3]:
headline = pd.read_pickle("../datastore/headline.p")
headlines = headline['headline'].tolist()

In [4]:
headlines

['지표로 보는 경제',
 '오늘의',
 '화제의 분양현장',
 '뉴스룸',
 '카드뉴스',
 '조선일보',
 'TV조선',
 '특집',
 '이슈',
 'Weekly BIZ',
 '금주의',
 '사회공헌 Together',
 '신나는 공부',
 '포토',
 '리빙포인트',
 '경제계 인사',
 '경향포토',
 '오늘의 날씨']

## Select Categories
- 포함 : 경제, 문화, 건강, 과학, 사회, 정치, 스포츠
- 제외 : 종합, 정보없음, 인물, 사설

In [5]:
train = pd.merge(train, category, on='category', how='inner')

In [6]:
categories = ['경제', '과학', '사회', '정치']

In [7]:
train = train[train['name'].isin(categories)]
train.name.unique()

array(['사회', '정치', '경제', '과학'], dtype=object)

## Preprocessing
1. Datetime (16-09-11 ~ 16-09-17)
2. Remove stopwords (regex, hanja)
3. POS Tagging with KoNLPy, Mecab
4. Using bigram

In [8]:
import datetime
from konlpy.tag import Mecab
import hanja
import re

In [9]:
mecab = Mecab()

In [10]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', ' ', text)
    for headline in headlines:
        text = text.replace(headline, ' ')
    return text

In [11]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [12]:
train['title_flat'] = train['title'].apply(lambda text: text_cleaning(text))
title = [tokenize(each[1]['title_flat']) for each in train.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [14]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)

In [15]:
x_list_100d = PCA(n_components=100).fit_transform(x_list.toarray())
x_list_100d.shape

(2802, 100)

### Scoring

In [16]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [17]:
best_score = 0.0
best_k = 0

In [18]:
for k in range(15, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_100d)
    score = silhouette_score(x_list_100d, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 15 , Score is : 0.171
In Clusters = 16 , Score is : 0.269
In Clusters = 17 , Score is : 0.200
In Clusters = 18 , Score is : 0.180
In Clusters = 19 , Score is : 0.184
In Clusters = 20 , Score is : 0.152
In Clusters = 21 , Score is : 0.174
In Clusters = 22 , Score is : 0.183
In Clusters = 23 , Score is : 0.122
In Clusters = 24 , Score is : 0.204
In Clusters = 25 , Score is : 0.241
In Clusters = 26 , Score is : 0.204
In Clusters = 27 , Score is : 0.093
In Clusters = 28 , Score is : 0.201
In Clusters = 29 , Score is : 0.134
In Clusters = 30 , Score is : 0.194
In Clusters = 31 , Score is : 0.207
In Clusters = 32 , Score is : 0.142
In Clusters = 33 , Score is : 0.173
In Clusters = 34 , Score is : 0.121
In Clusters = 16 , Best score is : 0.269


### K-Means Algorithm

In [19]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list_100d)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

326.517656329


In [20]:
x_list_vector = x_list_100d.tolist()
train = train.drop(['title_flat', 'target_str'], axis=1)
train['cluster'] = labels

## Choose Best Cluster
1. Cluster size < 500
2. Recent published
3. Minimum inertia

### Compare best cluster

In [21]:
sample_silhouette_values = silhouette_samples(x_list_100d, labels)
sample_silhouette_score = []
best_cluster = []
cluster_num = best_k

for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    print('Cluster %d: ' % (i), abs(ith_cluster_silhouette_values.mean()))
    sample_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

sample_silhouette_score.sort(reverse=True)
sample_silhouette_score = sample_silhouette_score[:8]

Cluster 0:  0.155573887263
Cluster 1:  0.117931453769
Cluster 2:  0.100998186721
Cluster 3:  0.317866191227
Cluster 4:  0.384792472186
Cluster 5:  0.106612674041
Cluster 6:  0.762580662685
Cluster 7:  0.18113818275
Cluster 8:  0.108352735955
Cluster 9:  0.313368064315
Cluster 10:  0.0378033411809
Cluster 11:  0.50333497927
Cluster 12:  0.215003388431
Cluster 13:  0.315701896084
Cluster 14:  0.999999995619
Cluster 15:  0.610743869363


In [22]:
sample_silhouette_score

[0.99999999561869757,
 0.76258066268477387,
 0.6107438693634909,
 0.50333497926984605,
 0.38479247218646612,
 0.31786619122715154,
 0.31570189608427901,
 0.31336806431538483]

In [23]:
for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i)

In [24]:
best_cluster

[3, 4, 6, 9, 11, 13, 14, 15]

In [25]:
train = train[train['cluster'].isin(best_cluster)]
train.cluster.unique()

array([13,  4,  3, 14, 11, 15,  6,  9])

## Result

In [26]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 10


,title,category
135,[경향포토]자리 앉는 야3당 원내대표,정치 > 정치일반
136,[경향포토]최순실 관련 야3당 원내대표 회동,정치 > 정치일반
228,[경향포토]긴급의총장 들어서는 우상호 원내대표,정치 > 정치일반
195,[경향포토]야3당 원내대표 회동,정치 > 정치일반
2767,10분만에 깨진 3黨 원내대표 회동,정치 > 국회ㆍ정당
131,[경향포토]손잡은 야3당 원내대표,정치 > 정치일반
137,[경향포토]야3당 원내대표 회동,정치 > 정치일반
196,[경향포토]야3당 원내대표 회의,정치 > 정치일반
83,[속보]국회의장·여야 원내대표 회동 10분 만에 결렬,정치 > 정치일반
194,[경향포토]야3당 회동,정치 > 정치일반





Cluster 1: Size 24


,title,category
292,[경향포토]국회 온 비서실장,정치 > 정치일반
211,[11·2 기습 개각]“비서실장 누가 맡으려 할까” 뒤엉켜버린 인적쇄신 수순,정치 > 정치일반
296,[경향포토]운영위 답변하는 한광옥 비서실장,정치 > 정치일반
1223,"김규현 외안수석, 청와대 비서실장 대행키로",정치 > 청와대
1238,"대통령 비서실장 한광옥, 정무수석 허원제",정치 > 청와대
1206,"비서실장도 못 구한채…박대통령, 황급히 ‘수족’ 쳐내",정치 > 청와대
1624,"[사설]한광옥 비서실장, 직언하지 못할 거면 시작도 말라",뉴스 > 정치 > 정치일반
297,[경향포토]국회 운영위 답변하는 한광옥 비서실장,정치 > 정치일반
964,"신임 대통령 비서실장에 DJ 비서실장, 한광옥 국민대통합위원장",정치 > 국회·정당
1649,15년 만에 다시 비서실장 기용된 한광옥,뉴스 > 정치 > 정치일반





Cluster 2: Size 6


...

,title,category
2088,"부동산 대책 앞두고… 정부, 건설업 리스크 요인 점검",뉴스 > 경제 > 부동산
1332,[기업특집]아모레퍼시픽 - ‘아시아의 미’ 집중 연구…국가별 피부색 맞춘 제품 개발,마켓·비즈 > 기업소식
1829,"팔 걷어붙인 오바마 ""공화국의 운명, 여러분에게 달려있다""",국제 > 미국ㆍ중남미
1152,"엘지 계열사 5곳, 탄소경영 최우수",경제 > 경제일반
1532,후임자 없이 참모 경질 처음… 위기 몰린 靑 ‘다급한 쇄신’,뉴스 > 정치 > 정치일반
1759,"檢·靑 압수수색 신경전… ""강제 진입"" 압박에 결국 손든 靑",사회 > 법원ㆍ검찰ㆍ경찰
633,채동욱 “눈치가 없어 법대로 하다 잘렸다”,사회 > 사회일반
376,"법원 “이승만 비판 ‘우남찬가’ 저자, 손해배상 책임 없어”",사회 > 사회일반
2513,[11·3 부동산 대책]과열 지역 위주 ‘급한 불 끄기’··· 재건축·청약광풍 부산...,부동산 >
1179,“내년 전국 주택 매맷값 0.8% 떨어질 듯”,경제 > 경제일반





Cluster 6: Size 5


,title,category
668,[경향포토]외국인 학생 패션쇼 - 5,사회 > 사회일반
669,[경향포토]외국인 학생 패션쇼 - 4,사회 > 사회일반
672,[경향포토]외국인 학생 패션쇼 - 1,사회 > 사회일반
671,[경향포토]외국인 학생 패션쇼 - 2,사회 > 사회일반
670,[경향포토]외국인 학생 패션쇼 - 3,사회 > 사회일반





Cluster 7: Size 4


,title,category
1453,[경향포토]호송차에서 내리는 최순실,사회 > 법원·검찰
1241,[경향포토]호송차에서 내리는 안종범 전 청와대 정책조정수석,정치 > 청와대
1460,[경향포토]호송차 오르는 최순실,사회 > 법원·검찰
703,[경향포토]호송차 향하는 최순실,사회 > 사회일반


## Save Dataframe to MongoDB

In [27]:
from pymongo import MongoClient

In [28]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')
articles = db.get_collection('articles')

In [29]:
articles.insert_many(train.to_dict(orient='records'))
client.close()